In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
from xgboost import XGBRegressor

from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, make_scorer
from sklearn.feature_selection import RFECV

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from scipy.stats import pearsonr

import matplotlib.pyplot as plt
import seaborn as sns

from itertools import product
from tqdm import tqdm
from time import sleep

In [2]:
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

rmse_scorer = make_scorer(rmse, greater_is_better=False)

In [3]:
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

In [4]:
SHyper = pd.read_csv('SCouping_product_smile.csv')

In [9]:
S_CmordredC = pd.read_csv('CatalystSC.csv')
S_CmordredC_fixes = S_CmordredC.dropna(axis=1, how='any')
S_CmordredC_fix = S_CmordredC_fixes.copy()
S_CmordredC_fix.drop(S_CmordredC_fix.columns[0], axis = 1, inplace = True)
S_CmordredC_fix.rename(columns=lambda x: x + "-1", inplace=True)

S_CmordredI = pd.read_csv('ImineSC.csv')
S_CmordredI_fixes = S_CmordredI.dropna(axis=1, how='any')
S_CmordredI_fix = S_CmordredI_fixes.copy()
S_CmordredI_fix.drop(S_CmordredI_fix.columns[0], axis = 1, inplace = True)
S_CmordredI_fix.rename(columns=lambda x: x + "-2", inplace=True)

S_CmordredT = pd.read_csv('ThiolSC.csv')
S_CmordredT_fixes = S_CmordredT.dropna(axis=1, how='any')
S_CmordredT_fix = S_CmordredT_fixes.copy()
S_CmordredT_fix.drop(S_CmordredT_fix.columns[0], axis = 1, inplace = True)
S_CmordredT_fix.rename(columns=lambda x: x + "-3", inplace=True)

S_CmordredP = pd.read_csv('ProductSC.csv')
S_CmordredP_fixes = S_CmordredP.dropna(axis=1, how='any')
S_CmordredP_fix = S_CmordredP_fixes.copy()
S_CmordredP_fix.drop(S_CmordredP_fix.columns[0], axis = 1, inplace = True)
S_CmordredP_fix.rename(columns=lambda x: x + "-4", inplace=True)

In [10]:
MordredPP = pd.concat([S_CmordredC_fix, S_CmordredI_fix, S_CmordredT_fix, S_CmordredP_fix], axis = 1)

In [12]:
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

In [13]:
mols1 = [Chem.MolFromSmiles(smi) for smi in SHyper['Catalyst']]
mols2 = [Chem.MolFromSmiles(smi) for smi in SHyper['Imine']]
mols3 = [Chem.MolFromSmiles(smi) for smi in SHyper['Thiol']]
mols4 = [Chem.MolFromSmiles(smi) for smi in SHyper['Product']]

In [14]:
from rdkit.Avalon import pyAvalonTools

Avs1 = [pyAvalonTools.GetAvalonFP(mol, nBits=2048) for mol in mols1]
df_Avs1 = pd.DataFrame(np.array(Avs1))
df_Avs1.columns = [f'1_{col}' for col in df_Avs1.columns]

Avs2 = [pyAvalonTools.GetAvalonFP(mol, nBits=2048) for mol in mols2]
df_Avs2 = pd.DataFrame(np.array(Avs2))
df_Avs2.columns = [f'2_{col}' for col in df_Avs2.columns]

Avs3 = [pyAvalonTools.GetAvalonFP(mol, nBits=2048) for mol in mols3]
df_Avs3 = pd.DataFrame(np.array(Avs3))
df_Avs3.columns = [f'3_{col}' for col in df_Avs3.columns]

Avs4 = [pyAvalonTools.GetAvalonFP(mol, nBits=2048) for mol in mols4]
df_Avs4 = pd.DataFrame(np.array(Avs4))
df_Avs4.columns = [f'4_{col}' for col in df_Avs4.columns]

In [51]:
Avs_PEMF = pd.concat([df_Avs1, df_Avs2, df_Avs3, df_Avs4], axis = 1)

In [52]:
SPP_columns = ['SMR_VSA9-1', 'nAromAtom-3']

In [53]:
SPP = MordredPP[SPP_columns]

In [54]:
SPP_Avs_PEMF = pd.concat([SPP, Avs_PEMF], axis = 1)

In [55]:
X = SPP_Avs_PEMF
y = SHyper.iloc[:,-2].copy()

In [58]:
X

,SMR_VSA9-1,nAromAtom-3,1_0,1_1,1_2,1_3,1_4,1_5,1_6,1_7,...,4_2038,4_2039,4_2040,4_2041,4_2042,4_2043,4_2044,4_2045,4_2046,4_2047
0,44.879733,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,44.879733,6,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,44.879733,6,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,44.879733,6,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,44.879733,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,34.124950,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1071,34.124950,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1072,34.124950,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1073,34.124950,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
y

0       1.152342
1       2.478059
2       1.578864
3       1.684091
4       1.487754
          ...   
1070   -0.240111
1071    0.191142
1072    0.530235
1073    0.166908
1074    0.023691
Name: Output, Length: 1075, dtype: float64

In [216]:
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=475, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42)

In [217]:
X_train_base, X_train_meta = X_train.iloc[:, 2:], X_train.iloc[:, :2]
X_val_base, X_val_meta = X_val.iloc[:, 2:], X_val.iloc[:, :2]
X_test_base, X_test_meta = X_test.iloc[:, 2:], X_test.iloc[:, :2]

In [218]:
param_grid_meta = {
    'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

In [219]:
XGBmodel = XGBRegressor(
    random_state=81,
    objective='reg:squarederror',   # 回归任务
    n_jobs=-1                       # 使用所有CPU核心
)

In [220]:
base_models = [
    ('rf', RandomForestRegressor(n_estimators=1500, random_state=81)),
    ('xgb', XGBRegressor(objective="reg:squarederror", n_estimators=10, learning_rate=0.1, max_depth=6, random_state=81)),
    ('et', ExtraTreesRegressor(n_estimators=600, random_state=81)),
    ('dt', DecisionTreeRegressor(max_depth = 10, min_samples_split = 6, random_state=81)),  # 添加决策树模型
    ('krr', KernelRidge(alpha=1.0, gamma=None))  # 添加核岭回归模型
]

In [221]:
kf = KFold(n_splits=5, shuffle=True, random_state=81)
meta_features_train = np.zeros((X_train_base.shape[0], len(base_models)))

for i, (name, model) in enumerate(base_models):
    print(f"Training base model: {name}")
    
    # 使用交叉验证生成元特征
    for train_index, val_index in kf.split(X_train_base):
        X_tr, X_val = X_train_base.iloc[train_index], X_train_base.iloc[val_index]
        y_tr = y_train.iloc[train_index]
        model.fit(X_tr, y_tr)
        meta_features_train[val_index, i] = model.predict(X_val)
    
    # 更新模型（在完整的训练集上重新训练）
    model.fit(X_train_base, y_train)

Training base model: rf
Training base model: xgb
Training base model: et
Training base model: dt
Training base model: krr


In [222]:
meta_features_val = np.zeros((X_val_base.shape[0], len(base_models)))
meta_features_test = np.zeros((X_test_base.shape[0], len(base_models)))

In [223]:
for i, (name, model) in enumerate(base_models):
    meta_features_val[:, i] = model.predict(X_val_base)
    meta_features_test[:, i] = model.predict(X_test_base)

In [224]:
meta_features_train = np.hstack([meta_features_train, X_train_meta.values])
meta_features_val = np.hstack([meta_features_val, X_val_meta.values])
meta_features_test = np.hstack([meta_features_test, X_test_meta.values])

In [225]:
meta_model = XGBRegressor(
    random_state=81,
    objective='reg:squarederror',   # 回归任务
    n_jobs=-1
)

In [226]:
grid_search_meta = GridSearchCV(
    estimator=meta_model,
    param_grid=param_grid,
    scoring=rmse_scorer,
    cv=3,
    verbose=2,
    n_jobs=-1
)

In [227]:
grid_search_meta.fit(meta_features_train, y_train)

Fitting 3 folds for each of 2268 candidates, totalling 6804 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 317 tasks      | elapsed:   49.4s
[Parallel(n_jobs=-1)]: Done 600 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 965 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 1410 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 1937 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 2544 tasks      | elapsed: 11.3min
[Parallel(n_jobs=-1)]: Done 3233 tasks      | elapsed: 15.0min
[Parallel(n_jobs=-1)]: Done 4002 tasks      | elapsed: 18.8min
[Parallel(n_jobs=-1)]: Done 4853 tasks      | elapsed: 22.3min
[Parallel(n_jobs=-1)]: Done 5784 tasks      | elapsed: 27.7min
[Parallel(n_jobs=-1)]: Done 6804 out of 6804 | elapsed: 33.1min finished


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=...
                                    tree_method=None, validate_parameters=None,
                                    verbosity=None),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0],
                         'learning_rate': [0.01, 0.05, 0.1],
            

In [228]:
print("Best Meta Model Parameters:", grid_search_meta.best_params_)
print("Best Meta Model RMSE:", -grid_search_meta.best_score_)

Best Meta Model Parameters: {'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.6}
Best Meta Model RMSE: 0.23220823841867635


In [229]:
best_meta_model = grid_search_meta.best_estimator_

In [230]:
y_val_pred = best_meta_model.predict(meta_features_val)
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f"Validation RMSE: {val_rmse:.4f}")

Validation RMSE: 0.2373


In [231]:
y_test_pred = best_meta_model.predict(meta_features_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f"Test RMSE: {test_rmse:.4f}")

Test RMSE: 0.2005


In [232]:
val_mae = mean_absolute_error(y_val, y_val_pred)
print(f"Validation MAE: {val_mae:.4f}")

Validation MAE: 0.1609


In [233]:
test_mae = mean_absolute_error(y_test, y_test_pred)
print(f"Test MAE: {test_mae:.4f}")

Test MAE: 0.1365


In [234]:
y_val_pred = best_meta_model.predict(meta_features_val)
val_r2 = r2_score(y_val, y_val_pred)
print(f"Validation R²: {val_r2:.4f}")

Validation R²: 0.8770


In [235]:
y_test_pred = best_meta_model.predict(meta_features_test)
test_r2 = r2_score(y_test, y_test_pred)
print(f"Test R²: {test_r2:.4f}")

Test R²: 0.9168
